# Modelagem e Simulação Multifísica

## Apresentação do curso e Introdução à modelagem Multifísica

### 1 - Exemplo de problemas multifíscos 

Nesta aula serão apresentados os problemas <a href="https://jsdokken.com/dolfinx-tutorial/chapter2/navierstokes.html"> A equação de Navier-Stokes </a> e Um sistema de equações de advecção-difusão, apresentados no livro "Solving PDEs in Python - The FEniCS Tutorial Volume I" de setembro de 2017. Espera-se que ao final do curso, o aluno seja capaz de identificar parte das estratégias usadas para a solução desses problemas, tais como: i) desenvolvimento de uma formulação fraca (ou variacional) para a representação de PDEs no Método de Elementos Finitos (FEM), ii) Solução de problemas não lineares e iii) ter uma noção de como abortdar problemas mutifísicos. 


### 2 -  A equação de Navier-Stokes

Esse exemplo aborda as equações de Navier-Stokes incompressíveis. Este problema combina muitos dos desafios a serem abordados na disciplina tais como dependência temporal, não linearidade e o uso de variáveis vetoriais. Embora muitos aspectos da solução do problema de Navier-Stokes estejam fora do escopo da disciplina, é possível observar que até mesmo um algoritmo relativamente complexo pode ser implementado com relativa facilidade no FEM.

#### 2.1 As equações governantes

As equações de Navier-Stokes incompressíveis formam um sistema de equações que relacionam a velocidade do fluido $u$ e sua pressão $p$:

\begin{eqnarray}
\rho\left(\frac{\partial \mathbf{u}}{\partial t} + \mathbf{u}\cdot\nabla\mathbf{u} \right) &=& \nabla\cdot\sigma(\mathbf{u},p)+f\\
\nabla\cdot\mathbf{u} &=& 0
\end{eqnarray}

onde $f$ é uma dada força por unidade de volume, $\rho$ a densidade do fluido, $\sigma(\mathbf{u},p)$ representa o tensor de tensão, que parar um fluido Newtoniano é dado por: 

\begin{equation}
\sigma(\mathbf{u},p) = 2\mu\epsilon(\mathbf{u}) - p\mathbf{I}
\end{equation}

onde $\mathbf{I}$ é uma matrix identidade com a mesma dimensão do problema, $\epsilon(\mathbf{u})$ é o tensor de deformação 
\begin{equation}
\epsilon(\mathbf{u}) = \frac{1}{2}\left(\nabla\mathbf{u} + (\nabla\mathbf{u}^T)\right)
\end{equation}

e o parâmtro $\mu$ é um escalar que representa a viscosidade dinâmica. Reparem que $\sigma$ e $\epsilon$ são grandezas tensoriais (representadas por matrizes NxN onde N é a dimensão do problema, i.e. 2D ou 3D).


#### 2.2 Solução de um problema bidimensional

Como será visto no curso, as equações de Navier-Stokes não podem ser inseridas diretamente no FEM. Alternativamente, iremos definir uma formulação conhecida como formulação fraca (ou forma variacional) que permitirá reescrever o problema de modo a ser facilmente incorporado ao FEM, que por sua vez resolve o novo problema equivalente com auxílio de uma malha que discretiza o domínio $\Omega$ do problema original.

Como exemplo, considere um fluido em um duto retangular que possui um obstácuo cilíndrico, conforme mostrado na figura a seguir

<center>Fig. 1 Geometria para o problema de teste do escoamento passado por um cilindro. Observe a geometria ligeiramente perturbada e assimétrica. </center>
<p>
    <img width="900" src="Figs/Navier-stokes_GEO-2d.png">
</p>


##### 2.2.1 Discretização da geometria

Usando o FEniCS, a geometria do problema pode ser construída usando alguns blocos simples (primitivas) e oprecações booleanas. Aternativamente, pode-se importar a geometria de diferentes softwares de Desenho Assistido por Computador (CAD). Nesse exemplo é usada uma API para o malhdor <a href="https://gmsh.info/"> GMSH </a> para gerar o retângulo e o círculo.

`gmsh`: biblioteca para gerar malhas.  
`os`: biblioteca para interação com o sistema de arquivos.  
`numpy`: biblioteca para manipulação de arrays e cálculos numéricos.  
`mpi4py`: biblioteca para comunicação entre processos paralelos.  
`dolfinx.io` e `dolfinx.mesh`: bibliotecas de funções para leitura, escrita e criação de malhas usando Dolfinx.  

`L` e `H`: dimensões do domínio.  
`c_x` e `c_y`: coordenadas do centro do obstáculo.  
`r`: raio do obstáculo.  
`gdim`: dimensão da malha.  
`mesh_comm`: objeto de comunicação entre processos.  
`model_rank`: define qual processo gerencia a criação da malha.  

In [3]:
import gmsh
import os
import numpy as np
#import tqdm.autonotebook
from mpi4py import MPI
from dolfinx.io import (VTXWriter, distribute_entity_data, gmshio) 
from dolfinx.mesh import create_mesh, meshtags_from_entities

gmsh.initialize()

L = 2.2
H = 0.41
c_x = c_y = 0.2
r = 0.05
gdim = 2  # Malha 2D
mesh_comm = MPI.COMM_WORLD
model_rank = 0

# Cria um retângulo representando a região do fluido e um disco para o obstáculo
if mesh_comm.rank == model_rank:
    rectangle = gmsh.model.occ.addRectangle(0, 0, 0, L, H, tag=1)
    obstacle = gmsh.model.occ.addDisk(c_x, c_y, 0, r, r)

ModuleNotFoundError: No module named 'numpy'

A próxima etapa é subtrair o obstáculo do canal, de modo que não se faça a malha no interior do círculo.

In [ ]:
# Subtrai o obstáculo do domínio do fluido e sincroniza a geometria
if mesh_comm.rank == model_rank:
    fluid = gmsh.model.occ.cut([(gdim, rectangle)], [(gdim, obstacle)])
    gmsh.model.occ.synchronize()

Para fazer com que o GMSH gere a malha do fluido, adicionamos um marcador de volume físico na região de interesse.

In [ ]:
# Define o marcador para o fluido
fluid_marker = 1
# Obtém os volumes da malha e define grupos físicos para o fluido
if mesh_comm.rank == model_rank:
    volumes = gmsh.model.getEntities(dim=gdim)
    assert (len(volumes) == 1)
    gmsh.model.addPhysicalGroup(volumes[0][0], [volumes[0][1]], fluid_marker)
    gmsh.model.setPhysicalName(volumes[0][0], fluid_marker, "Fluid")

Para marcar as diferentes superfícies da malha, atribuiremos o marcador 2 à entrada (lado esquerdo), o marcador 3 à saída (lado direito), o marcador 4 às paredes do fluido e o marcador 5 ao obstáculo. Faremos isso calculando o centro de massa de cada entidade geométrica.

In [ ]:
# Define marcadores para as condições de contorno: entrada, saída, paredes e obstáculo
inlet_marker, outlet_marker, wall_marker, obstacle_marker = 2, 3, 4, 5
inflow, outflow, walls, obstacle = [], [], [], []
if mesh_comm.rank == model_rank:
    boundaries = gmsh.model.getBoundary(volumes, oriented=False)
    for boundary in boundaries:
        center_of_mass = gmsh.model.occ.getCenterOfMass(boundary[0], boundary[1])
        if np.allclose(center_of_mass, [0, H / 2, 0]):
            inflow.append(boundary[1])
        elif np.allclose(center_of_mass, [L, H / 2, 0]):
            outflow.append(boundary[1])
        elif np.allclose(center_of_mass, [L / 2, H, 0]) or np.allclose(center_of_mass, [L / 2, 0, 0]):
            walls.append(boundary[1])
        else:
            obstacle.append(boundary[1])
    gmsh.model.addPhysicalGroup(1, walls, wall_marker)
    gmsh.model.setPhysicalName(1, wall_marker, "Walls")
    gmsh.model.addPhysicalGroup(1, inflow, inlet_marker)
    gmsh.model.setPhysicalName(1, inlet_marker, "Inlet")
    gmsh.model.addPhysicalGroup(1, outflow, outlet_marker)
    gmsh.model.setPhysicalName(1, outlet_marker, "Outlet")
    gmsh.model.addPhysicalGroup(1, obstacle, obstacle_marker)
    gmsh.model.setPhysicalName(1, obstacle_marker, "Obstacle")

Gerando a malha

In [ ]:
# Define o tamanho dos elementos da malha e a gera
gmsh.option.setNumber("Mesh.CharacteristicLengthMin", 0.0008)
gmsh.option.setNumber("Mesh.CharacteristicLengthMax", 0.04)
gmsh.model.mesh.generate(gdim)

# Converte a malha criada no GMSH para o formato utilizado pelo Dolfinx
mesh, _, ft = gmshio.model_to_mesh(gmsh.model, mesh_comm, model_rank, gdim=gdim)
ft.name = "Facet markers"
# Finaliza o GMSH
gmsh.finalize()

In [ ]:
# Configura a visualização com PyVista
from dolfinx.plot import vtk_mesh
import pyvista
pyvista.start_xvfb()

# Extrai a topologia da malha para criar uma visualização em PyVista
topology, cell_types, x = vtk_mesh(mesh)
grid = pyvista.UnstructuredGrid(topology, cell_types, x)

# Configura e exibe a malha com a visualização 2D
plotter = pyvista.Plotter();
plotter.add_mesh(grid, show_edges=True);
plotter.window_size = [1000, 250];
plotter.camera_position = 'xy'
plotter.camera.zoom(3)
plotter.show();

##### 2.2.1 Parâmetros inicias e condições de contorno

Além das propriedades do fluido, é necessário definir as condições inicias e as condições de contorno nas fronteiras do domínio. Reparem que a superfície do cilindro faz parte da fronteira externa do domínio.  
Parâmetros do problema:

In [ ]:
# Interface Python para PETSc (Portable, Extensible Toolkit for Scientific Computation), 
# uma biblioteca usada para resolver sistemas lineares e não-lineares.
# PETSc é utilizado principalmente para resolver equações diferenciais parciais (PDEs)
from petsc4py import PETSc

# Importa funções específicas do módulo fem (Finite Element Method) do DOLFINx
# Constant: Define constantes no domínio do problema
# Function: Representa uma função que vive em um espaço de funções (pode ser usada para representar soluções ou parâmetros).
# functionspace: Define o espaço de funções onde as soluções serão encontradas
# assemble_scalar: Monta uma expressão escalar a partir de uma formulação variacional.
# dirichletbc: Define condições de contorno de Dirichlet para o problema.
# form: Constrói uma forma variacional a partir de uma expressão simbólica.
# locate_dofs_topological: Encontra graus de liberdade (DOFs) de acordo com a topologia do domínio (como malhas).
# set_bc: Define condições de contorno.
from dolfinx.fem import (Constant, Function, functionspace, 
                         assemble_scalar, dirichletbc, form, locate_dofs_topological, set_bc)

# Parâmetros iniciais
t = 0
T = 2.0 #5.0 # final time
num_steps = 2000 # number of time steps
dt = T / num_steps # time step size
k = Constant(mesh, PETSc.ScalarType(dt))

# Define propriedades físicas do fluido: viscosidade dinâmica e densidade
mu = Constant(mesh, PETSc.ScalarType(0.001))  # Dynamic viscosity
rho = Constant(mesh, PETSc.ScalarType(1))     # Density

Condições de contorno

In [ ]:
from basix.ufl import element

# Define os espaços de função para a velocidade (V) e a pressão (Q)
v_cg2 = element("Lagrange", mesh.topology.cell_name(), 2, shape=(mesh.geometry.dim, ))
s_cg1 = element("Lagrange", mesh.topology.cell_name(), 1)
V = functionspace(mesh, v_cg2)
Q = functionspace(mesh, s_cg1)

fdim = mesh.topology.dim - 1

# Classe para definir o perfil de velocidade na entrada do domínio - Define condições de fronteira
class InletVelocity():
    def __init__(self, t):
        self.t = t

    def __call__(self, x):
        values = np.zeros((gdim, x.shape[1]), dtype=PETSc.ScalarType)
        #values[0] = 4 * 1.5 * np.sin(self.t * np.pi / 8) * x[1] * (0.41 - x[1]) / (0.41**2)
        values[0] = 4 * 1.5 * x[1] * (0.41 - x[1]) / (0.41**2)
        return values


# Inlet
# Interpola o perfil de velocidade de entrada no espaço de funções de velocidade
u_inlet = Function(V)
inlet_velocity = InletVelocity(t)
u_inlet.interpolate(inlet_velocity)
bcu_inflow = dirichletbc(u_inlet, locate_dofs_topological(V, fdim, ft.find(inlet_marker)))
# Walls
u_nonslip = np.array((0,) * mesh.geometry.dim, dtype=PETSc.ScalarType)
bcu_walls = dirichletbc(u_nonslip, locate_dofs_topological(V, fdim, ft.find(wall_marker)), V)
# Obstacle
bcu_obstacle = dirichletbc(u_nonslip, locate_dofs_topological(V, fdim, ft.find(obstacle_marker)), V)
bcu = [bcu_inflow, bcu_obstacle, bcu_walls]
# Outlet
bcp_outlet = dirichletbc(PETSc.ScalarType(0), locate_dofs_topological(Q, fdim, ft.find(outlet_marker)), Q)
bcp = [bcp_outlet]

##### 2.2.2 Implementando o FEM usando o FEniCS.

In [ ]:
# dolfinx.cpp.mesh: Lida com operações avançadas relacionadas à malha,
# como determinar tipos de células e entidades.
from dolfinx.cpp.mesh import to_type, cell_entity_type
# apply_lifting, assemble_matrix, assemble_vector, etc.:
# Funções PETSc relacionadas à montagem e aplicação de matrizes e vetores,
# essenciais para a resolução de sistemas de equações que descrevem o problema físico
from dolfinx.fem.petsc import (apply_lifting, assemble_matrix, assemble_vector,
                               create_vector, create_matrix, set_bc)
from dolfinx.graph import adjacencylist
# bb_tree, compute_collisions_points, compute_colliding_cells: Funções geométricas que ajudam a detectar 
# colisões ou interseções entre pontos e células da malha
from dolfinx.geometry import bb_tree, compute_collisions_points, compute_colliding_cells

from ufl import (FacetNormal, Identity, Measure, TestFunction, TrialFunction,
                 as_vector, div, dot, ds, dx, inner, lhs, grad, nabla_grad, rhs, sym, system)


# Define funções de teste e variáveis para a solução - Define trial and test functions
# TrialFunction e TestFunction: Definem as funções de teste e tentativa que fazem parte da formulação de elementos finitos. 
# Essas funções são usadas para montar a forma fraca das equações diferenciais.
# O espaço V é o espaço de funções para a variável de velocidade, 
# e o espaço Q é o espaço para a variável de pressão
# u e p são as funções desconhecidas (soluções que estamos buscando)
# v e q são as funções de teste (usadas na formulação variacional para gerar o sistema de equações)
u = TrialFunction(V)
v = TestFunction(V)
p = TrialFunction(Q)
q = TestFunction(Q)

# Define functions for solutions at previous and current time steps
u_ = Function(V)
u_.name = "u"
u_s = Function(V)
u_n = Function(V)
u_n1 = Function(V)
p_ = Function(Q)
p_.name = "p"
phi = Function(Q)

# Define variational problem for step 1
f = Constant(mesh, PETSc.ScalarType((0, 0)))
F1 = rho / k * dot(u - u_n, v) * dx
F1 += inner(dot(1.5 * u_n - 0.5 * u_n1, 0.5 * nabla_grad(u + u_n)), v) * dx
F1 += 0.5 * mu * inner(grad(u + u_n), grad(v)) * dx - dot(p_, div(v)) * dx
F1 += dot(f, v) * dx
a1 = form(lhs(F1))
L1 = form(rhs(F1))
A1 = create_matrix(a1)
b1 = create_vector(L1)

# Define variational problem for step 2
a2 = form(dot(grad(p), grad(q)) * dx)
L2 = form(-rho / k * dot(div(u_s), q) * dx)
A2 = assemble_matrix(a2, bcs=bcp)
A2.assemble()
b2 = create_vector(L2)

# Define variational problem for step 3
a3 = form(rho * dot(u, v) * dx)
L3 = form(rho * dot(u_s, v) * dx - k * dot(nabla_grad(phi), v) * dx)
A3 = assemble_matrix(a3)
A3.assemble()
b3 = create_vector(L3)

#saving data
from pathlib import Path
folder = Path("results")
folder.mkdir(exist_ok=True, parents=True)
vtx_u = VTXWriter(mesh.comm, folder / "Aula00_u.bp", [u_])
vtx_p = VTXWriter(mesh.comm, folder / "Aula00_p.bp", [p_])
vtx_u.write(t)
vtx_p.write(t)
#progress = tqdm.autonotebook.tqdm(desc="Solving PDE", total=num_steps)

# Configura o primeiro resolvedor para a etapa de velocidade - Solver Configuration for step 1
solver1 = PETSc.KSP().create(mesh.comm)
solver1.setOperators(A1)
solver1.setType(PETSc.KSP.Type.BCGS)
pc1 = solver1.getPC()
pc1.setType(PETSc.PC.Type.JACOBI)

# Solver Configuration for step 2
solver2 = PETSc.KSP().create(mesh.comm)
solver2.setOperators(A2)
solver2.setType(PETSc.KSP.Type.MINRES)
pc2 = solver2.getPC()
pc2.setType(PETSc.PC.Type.HYPRE)
pc2.setHYPREType("boomeramg")

# Solver Configuration for step 3
solver3 = PETSc.KSP().create(mesh.comm)
solver3.setOperators(A3)
solver3.setType(PETSc.KSP.Type.CG)
pc3 = solver3.getPC()
pc3.setType(PETSc.PC.Type.SOR)

In [ ]:
from IPython.display import clear_output # To clear the console at each iteration 

for i in range(num_steps):
    print(f"Iteration: {i} out of {num_steps}")

    #progress.update(1)
    # Update current time step
    t += dt
    # Update inlet velocity
    #inlet_velocity.t = t
    #u_inlet.interpolate(inlet_velocity)

    # Step 1: Tentative velocity step
    A1.zeroEntries()
    assemble_matrix(A1, a1, bcs=bcu)
    A1.assemble()
    with b1.localForm() as loc:
        loc.set(0)
    assemble_vector(b1, L1)
    apply_lifting(b1, [a1], [bcu])
    b1.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
    set_bc(b1, bcu)
    solver1.solve(b1, u_s.vector)
    u_s.x.scatter_forward()

    # Step 2: Pressure corrrection step
    with b2.localForm() as loc:
        loc.set(0)
    assemble_vector(b2, L2)
    apply_lifting(b2, [a2], [bcp])
    b2.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
    set_bc(b2, bcp)
    solver2.solve(b2, phi.vector)
    phi.x.scatter_forward()

    p_.vector.axpy(1, phi.vector)
    p_.x.scatter_forward()

    # Step 3: Velocity correction step
    with b3.localForm() as loc:
        loc.set(0)
    assemble_vector(b3, L3)
    b3.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
    solver3.solve(b3, u_.vector)
    u_.x.scatter_forward()

    # Write solutions to file
    vtx_u.write(t)
    vtx_p.write(t)

    # Update variable with solution form this time step
    with u_.vector.localForm() as loc_, u_n.vector.localForm() as loc_n, u_n1.vector.localForm() as loc_n1:
        loc_n.copy(loc_n1)
        loc_.copy(loc_n)

    clear_output(wait=True)

vtx_u.close()
vtx_p.close()

#### 2.3 Resultados.

In [ ]:
pyvista.start_xvfb()
topology, cell_types, geometry = vtk_mesh(V)
values = np.zeros((geometry.shape[0], 3), dtype=np.float64)
values[:, :len(u_n)] = u_n.x.array.real.reshape((geometry.shape[0], len(u_n)))

# Create a point cloud of glyphs
function_grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)
function_grid["u"] = values
glyphs = function_grid.glyph(orient="u", factor=0.1)

# Create a pyvista-grid for the mesh
mesh.topology.create_connectivity(mesh.topology.dim, mesh.topology.dim)
grid = pyvista.UnstructuredGrid(*vtk_mesh(mesh, mesh.topology.dim))

# Create plotter
plotter = pyvista.Plotter()
plotter.add_mesh(grid, style="wireframe", color="k")
plotter.add_mesh(glyphs)
plotter.view_xy()
plotter.window_size = [1000, 250];
plotter.camera.zoom(3)
plotter.show()

### 3 - Problemas Multifísicos:

Na maioria das disciplinas do curso de engenharia, nos vemos envolvidos com problemas expressos por uma única PDE, seja ela escalar ou vetorial. Alguns exemplos são os problemas regidos pela equação de Poisson (Problemas Eletrostáticos, Magnetostáticos, Elastostáticos, etc) ou problemas regidos pela equação de Onda (Problesas Acústicos, Eletromagnéticos). Entretanto, em problemas reais de engenharia, muitas vezes os problemas são expressos como um sistema de PDEs. Geralmente, cada PDE descreve diferentes quantidades que são regidas por físicas (muito) diferentes. 

Entender como diferentes físicas interagem em um problema multifísico é importante para se determinar o tipo de solução para o problema. Imagine que seja necessário avaliar o comportamento elétrico e térmico de um determinado circuito. Se considerarmos que a resistividade dos materiais não depende da temperatura dos componentes, podemos resolver primeiramente o problema elétrico e, posteriormente, usar a potência dissipada como fonte para o problema térmico. Nesse caso, o problema multifísico pode ser considerado um problema desacoplado. Em problemas ditos desacoplados, a iteração entre as físicas é mínima e o problema pode ser dividio em subproblemas independentes.   

Entretanto, se a temperatura varia de tal modo que ela afeta a condutividade dos componentes, o problema elétrico passa a ser dependente da temperatura e os dois problemas não podem mais ser resolvidos separadamente. Felizmente, em problemas termo-elétricos, as constantes de tempo do problema elétrico costumam ser bem mais rápidas do que as variações de temperatura e o acoplamento entre as físicas é considerado fraco. 

Problemas fracamente acoplados geralmente podem ser resolvidos por meio de processos iterativos. Um esquema tradicional que implementa esse acoplamento fraco pode ser descrito como: Primeiramente, o problema elétrico é solucionado e a densidade de potênca dissipada na forma de calor nos componentes do circuito é calculada. Posteriormente, essa densidade de potência é usada como a fonte do problema térmico, onde a elevação de temperatura é obtida para alguns passos de tempo. Durante esse processo, as propriedades elétricas são mantidas constantes. Uma vez obtida a nova temperatura, as propriedades elétricas dos componentes do circuito são recalculadas e o problema elétrico é atualizado. O ciclo é repetido até um critério de parada pré-estabelecido. 

A solução de problemas multifísicos fracamente acoplados é relativamente simples e intuitiva. Entretanto, tais processos iterativos podem ser sensíveis à escolhas de parâmetros, como por exemplo, qual o melhor momento para atualizar o problema elétrico? Atualizar o problema elétrico a cada passo do problema térmico pode levar a um processo computacionalmente ineficiente. Por outro lado, propagar a temperatura por longos períodos sem atualizar os parâmetros elétricos pode levar a resultados imprecisos. 

Por fim, quando o problema é fortemente acoplado, as equações governantes devem ser agrupadas em um único sistema de PDEs. Na próxima seção, veremos como o FEniCX pode ser usado para escrever solucionadores para esses sistemas de PDEs acoplados. O objetivo é demonstrar como é fácil implementar solucionadores totalmente implícitos, também conhecidos como monolíticos. Tais soluções funcionam bem quando as constantes de tempos das várias físicas tem ordem de grandeza similares. Caso contrário, o sistema resultante pode ser de difícil solução ou de custo computacional proibitivo.são

### 4 Um sistema de equações de advecção-difusão

Considere o seguinte sistema de equações de advecção-difusão:

\begin{eqnarray}
\frac{\partial u_1}{\partial t} + \mathbf{w}\cdot\nabla u_1 -  \nabla\cdot(\epsilon\nabla u_1) &=& f_1 -Ku_1u_2 \\
\frac{\partial u_2}{\partial t} + \mathbf{w}\cdot\nabla u_2 -  \nabla\cdot(\epsilon\nabla u_2) &=& f_2 -Ku_1u_2 \\
\frac{\partial u_3}{\partial t} + \mathbf{w}\cdot\nabla u_3 -  \nabla\cdot(\epsilon\nabla u_3) &=& f_3 +Ku_1u_2 - +Ku_3
\end{eqnarray}

Este sistema modela a reação química entre duas espécies A e B em algum domínio $\Omega$:
\begin{equation}
A+B \rightarrow C.
\end{equation}

Assumimos que a reação é de primeira ordem, o que significa que a taxa de reação é proporcional às concentrações $[A]$ e $[B]$ das duas espécies $A$ e $B$:
\begin{equation}
\frac{d}{dt}[C] = K[A][B]
\end{equation}

Assumimos também que a espécie formada $C$ decai espontaneamente com uma taxa proporcional à concentração $[C]$. No sistema de EDPs acima, usamos as variáveis $u_1 = [A]$, $u_2 = [B]$ e $u_3 = [C]$ para denotar as concentrações das três espécies. Observamos que as reações químicas são consideradas no lado direito do sistema de EDPs. As reações químicas ocorrem em cada ponto do domínio $\Omega$. Além disso, assumimos que as espécies $A$, $B$ e $C$ se difundem por todo o domínio com difusividade $\epsilon$. Os dois termos no lado esquerdo das PDEs representam processos físicos diferentes: o termo $\nabla\cdot(\epsilon\nabla u)$ corresponde a difusão normal enquanto o termo  $w\cdot\nabla u$ descreve convecção ou advecção, onde $w$ é a velocidade do fluido. 

Para tornar as coisas visual e fisicamente interessantes, deixaremos a reação química ocorrer no campo de velocidade calculado a partir da solução das equações de Navier-Stokes incompressíveis ao redor de um cilindro da seção anterior. Em resumo, estaremos resolvendo o seguinte sistema acpplado de EDPs não lineares:

\begin{eqnarray}
\rho\left(\frac{\partial \mathbf{w}}{\partial t} + \mathbf{w}\cdot\nabla\mathbf{w} \right) &=& \nabla\cdot\sigma(\mathbf{w},p)+f\\
\nabla\cdot\mathbf{w} &=& 0 \\
\frac{\partial u_1}{\partial t} + \mathbf{w}\cdot\nabla u_1 -  \nabla\cdot(\epsilon\nabla u_1) &=& f_1 -Ku_1u_2 \\
\frac{\partial u_2}{\partial t} + \mathbf{w}\cdot\nabla u_2 -  \nabla\cdot(\epsilon\nabla u_2) &=& f_2 -Ku_1u_2 \\
\frac{\partial u_3}{\partial t} + \mathbf{w}\cdot\nabla u_3 -  \nabla\cdot(\epsilon\nabla u_3) &=& f_3 +Ku_1u_2 - +Ku_3
\end{eqnarray}




#### 4.1 Solução usando o Método de Elementos Finitos

Não entraremos muito nos detalhes da solução desse problema. Recomenda-se que ao final do curso o aluno retorne ao problema 3.5 apresentado no livro "Solving PDEs in Python - The FEniCS Tutorial" para avaliar seu domínio sobre todos os passos utilizados na solução desse problema. Entretanto, é importante observar alguns detalhes na solução do problema: i) O problema não resolve simultaneamente as cinco equações. Apenas as três equações de advecção-difusão são resolvidas simultaneamente. ii) O problema assume que a valiação das concentrações das espécies não tem impacto na densidade e velocidade do fluido. Como consequência, os resultados do problema anterior podem ser reaproveitados. As equações de Naveir-Stokes podem ser tratadas como um problema desacoplado e o campom vetorial $\mathbf{w}$, obtido no problema anterior, pode ser considerado conhecido e aplicado diretamente nas equações de advecção-difusão. 

O código para a solução do problema é apresentado a seguir:

In [ ]:
from ufl import split

from dolfinx.fem.petsc import NonlinearProblem
from dolfinx.nls.petsc import NewtonSolver
from basix.ufl import mixed_element

#initial parameters
#The final time (T) and the num_steps should be the same for the Navier-Stokes problem
dt = T / num_steps # time step size

#reaction parameters
eps = Constant(mesh, PETSc.ScalarType(0.01)) # diffusion coefficient
K = Constant(mesh, PETSc.ScalarType(10.0))  # reaction rate

# Define function space for system of concentrations
S_el = mixed_element([s_cg1, s_cg1, s_cg1])
V3 = functionspace(mesh, S_el)

# Define test functions
v_1, v_2, v_3 = TestFunction(V3)

# Define functions for velocity and concentrations
w = Function(V)

u = Function(V3)
u_n = Function(V3)
# Split system functions to access components
u_1, u_2, u_3 = split(u)
u_n1, u_n2, u_n3 = split(u_n)

# Define source terms
class Mysource:
    def __init__(self, x0, x1):
        self.x0 = x0
        self.x1 = x1
    
    def __call__(self, x):
        values = np.where((x[0] - self.x0)**2 + (x[1] - self.x1)**2 < 0.05**2, .1, 0)
        return values

# f
f = Function(V3)
mysource1 = Mysource(.1,.1)
mysource2 = Mysource(.1,.3)
f.sub(0).interpolate(mysource1)
f.sub(1).interpolate(mysource2)

f_1, f_2, f_3 = split(f)

# Define expressions used in variational forms
k = Constant(mesh, PETSc.ScalarType(dt))

# Define variational problem
F = ((u_1 - u_n1) / k)*v_1*dx + dot(w, grad(u_1))*v_1*dx \
    + eps*dot(grad(u_1), grad(v_1))*dx + K*u_1*u_2*v_1*dx \
    + ((u_2 - u_n2) / k)*v_2*dx + dot(w, grad(u_2))*v_2*dx \
    + eps*dot(grad(u_2), grad(v_2))*dx + K*u_1*u_2*v_2*dx \
    + ((u_3 - u_n3) / k)*v_3*dx + dot(w, grad(u_3))*v_3*dx \
    + eps*dot(grad(u_3), grad(v_3))*dx - K*u_1*u_2*v_3*dx + K*u_3*v_3*dx \
    - f_1*v_1*dx - f_2*v_2*dx - f_3*v_3*dx

# Create time series for reading velocity data
#timeseries_w = TimeSeries('navier_stokes_cylinder/velocity_series')

# Time-stepping
t = 0

# Get velocity from previous problem
#with u_.vector.localForm() as loc_, w.vector.localForm() as loc_w:
#    loc_w.copy(loc_)
    
w.x.array[:] = u_.x.array

for n in range(num_steps):
    print(f"Iteration: {n} out of {num_steps}")
    # Update current time
    t += dt
    # Read velocity from file
    #timeseries_w.retrieve(w.vector(), t)
    
    # Solve variational problem for time step
    problem = NonlinearProblem(F, u)
    solver = NewtonSolver(MPI.COMM_WORLD, problem)
    solver.solve(u)
    u.x.scatter_forward()
    
    clear_output(wait=True)
    
    # Update previous solution
    u_n.x.array[:] = u.x.array


#### 4.2 Resultados

In [ ]:
u1, u2, u3 = u.sub(0).collapse(), u.sub(1).collapse(), u.sub(2).collapse()

u_topology, u_cell_types, u_geometry = vtk_mesh(Q)
u_grid = pyvista.UnstructuredGrid(u_topology, u_cell_types, u_geometry)

u_grid.point_data["Concentrations of the specie [A]"] = u1.x.array.real
u_grid.set_active_scalars("Concentrations of the specie [A]")
u_plotter = pyvista.Plotter()
u_plotter.add_mesh(u_grid, show_edges=True)
plotter.window_size = [600, 100];
plotter.camera.zoom(3)
u_plotter.view_xy()
u_plotter.show()

u_grid.point_data["Concentrations of the specie [B]"] = u2.x.array.real
u_grid.set_active_scalars("Concentrations of the specie [B]")
u_plotter = pyvista.Plotter()
u_plotter.add_mesh(u_grid, show_edges=True)
plotter.window_size = [600, 100];
plotter.camera.zoom(3)
u_plotter.view_xy()
u_plotter.show()
    
u_grid.point_data["Concentrations of the specie [C]"] = u3.x.array.real
u_grid.set_active_scalars("Concentrations of the specie [C]")
u_plotter = pyvista.Plotter()
u_plotter.add_mesh(u_grid, show_edges=True)
plotter.window_size = [600, 100];
plotter.camera.zoom(3)
u_plotter.view_xy()
u_plotter.show()

In [ ]:
import shutil # To remove the navier_stokes_cylinder folder at the end of the simulation
shutil.rmtree("results")